In [2]:
import typing

class CountVectorizer():
    
    def __init__(self,lowercase=True):
        self.lowercase = lowercase
        
    def fit_transform(self,lst) -> list:
        '''
        Transform sentences to matrix_word
        '''
        dict_ = {}
        
        #getting dict {word:{row:count}}
        for i,el in enumerate(lst): 
            lst_word = el.split(' ')
            for word in lst_word:
                word = word.lower()
                if word not in dict_:
                    dict_[word] = dict()
                    dict_[word][i] = 1
                elif i in dict_[word]:
                    dict_[word][i] = dict_[word][i] + 1
                else:
                    dict_[word][i] = 1
        
        #final matrix
        lst_ = []
        #initialization
        for i in range(len(lst)):
            lst_.append([])
            for j in range(len(dict_)):
                lst_[i].append(0)

        for i,word in enumerate(list(dict_.keys())):
            for k1,v1 in dict_[word].items():
                lst_[k1][i] = v1
        self.features_list = dict_.keys() # сразу сделать лист
        
        #посмотреть Counter Collections defaultdict setdefault
        
        return lst_
    
    def get_feature_names(self) -> list:
        '''
        Getting list of words
        '''
        return list(self.features_list)
    
    #def tf_transform(self):
     #   tf_matrix = [[ num / sum(row) for num in row] for row in matrix]
            


In [6]:
corpus = [
 'Crock Pot Pasta Never boil pasta again',
 'Pasta Pomodoro Fresh ingredients Parmesan to taste'
]
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names())
print(count_matrix)
print(vectorizer.get_feature_names())
print(sum([sum(row) for row in count_matrix]))


[[1, 1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]]
['crock', 'pot', 'pasta', 'never', 'boil', 'again', 'pomodoro', 'fresh', 'ingredients', 'parmesan', 'to', 'taste']
14


In [11]:
def tf_transform(matrix):
    total = sum([sum(row) for row in count_matrix])
    return [[round(num / total,2) for num in row] for row in matrix]

In [12]:
tf_transform(count_matrix)

[[0.07, 0.07, 0.14, 0.07, 0.07, 0.07, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.07, 0.0, 0.0, 0.0, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07]]

In [37]:
from math import log
def idf_transform(matrix):
    doc_cnt = len(matrix)
    matrix_idf = []
    for i,doc in enumerate(matrix):
        for j,num in enumerate(doc):
            if i == 0:
                matrix_idf.append(min(num,1))
            else:
                matrix_idf[j] = matrix_idf[j]+min(num,1)
    
    return [log((elem+1) / (doc_cnt+1)) + 1 for elem in matrix_idf]

In [38]:
idf_transform(count_matrix)

[0.5945348918918356,
 0.5945348918918356,
 1.0,
 0.5945348918918356,
 0.5945348918918356,
 0.5945348918918356,
 0.5945348918918356,
 0.5945348918918356,
 0.5945348918918356,
 0.5945348918918356,
 0.5945348918918356,
 0.5945348918918356]

In [76]:
def fit_transform(matrix):
    print (tf_transform(matrix))
    print(idf_transform(matrix))
    return [[tf*idf for tf,idf in zip(tf_row,idf_transform(matrix))] for tf_row in tf_transform(matrix)]
        
        
            

In [77]:
fit_transform(count_matrix)

[[0.07, 0.07, 0.14, 0.07, 0.07, 0.07, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.07, 0.0, 0.0, 0.0, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07]]
[0.5945348918918356, 0.5945348918918356, 1.0, 0.5945348918918356, 0.5945348918918356, 0.5945348918918356, 0.5945348918918356, 0.5945348918918356, 0.5945348918918356, 0.5945348918918356, 0.5945348918918356, 0.5945348918918356]


[[0.0416174424324285,
  0.0416174424324285,
  0.14,
  0.0416174424324285,
  0.0416174424324285,
  0.0416174424324285,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.07,
  0.0,
  0.0,
  0.0,
  0.0416174424324285,
  0.0416174424324285,
  0.0416174424324285,
  0.0416174424324285,
  0.0416174424324285,
  0.0416174424324285]]

In [81]:
class TfidfTransformer():
    
    def __init__(self,lowercase=True):
        self.lowercase = lowercase
    
    def tf_transform(self,matrix):
        total = sum([sum(row) for row in count_matrix])
        return [[round(num / total,2) for num in row] for row in matrix]
    
    def idf_transform(self,matrix):
        doc_cnt = len(matrix)
        matrix_idf = []
        for i,doc in enumerate(matrix):
            for j,num in enumerate(doc):
                if i == 0:
                    matrix_idf.append(min(num,1))
                else:
                    matrix_idf[j] = matrix_idf[j]+min(num,1)
    
        return [log((elem+1) / (doc_cnt+1)) + 1 for elem in matrix_idf]
    
    def fit_transform(self,matrix):
        return [[tf*idf for tf,idf in zip(tf_row,self.idf_transform(matrix))] for tf_row in self.tf_transform(matrix)]

In [83]:
transformer = TfidfTransformer()
tfidf_matrix = transformer.fit_transform(count_matrix)
print(tfidf_matrix)

[[0.0416174424324285, 0.0416174424324285, 0.14, 0.0416174424324285, 0.0416174424324285, 0.0416174424324285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.07, 0.0, 0.0, 0.0, 0.0416174424324285, 0.0416174424324285, 0.0416174424324285, 0.0416174424324285, 0.0416174424324285, 0.0416174424324285]]


In [87]:
class TfidfVectorizer():
    
    def __init__(self,lowercase=True):
        self.lowercase = lowercase
    
    def tf_transform(self,matrix):
        total = sum([sum(row) for row in count_matrix])
        return [[round(num / total,2) for num in row] for row in matrix]
    
    def idf_transform(self,matrix):
        doc_cnt = len(matrix)
        matrix_idf = []
        for i,doc in enumerate(matrix):
            for j,num in enumerate(doc):
                if i == 0:
                    matrix_idf.append(min(num,1))
                else:
                    matrix_idf[j] = matrix_idf[j]+min(num,1)
    
        return [log((elem+1) / (doc_cnt+1)) + 1 for elem in matrix_idf]
    
    def fit_transform(self,lst):
        
        '''
        Transform sentences to matrix_word
        '''
        dict_ = {}
        
        #getting dict {word:{row:count}}
        for i,el in enumerate(lst): 
            lst_word = el.split(' ')
            for word in lst_word:
                word = word.lower()
                if word not in dict_:
                    dict_[word] = dict()
                    dict_[word][i] = 1
                elif i in dict_[word]:
                    dict_[word][i] = dict_[word][i] + 1
                else:
                    dict_[word][i] = 1
        
        #final matrix
        lst_ = []
        #initialization
        for i in range(len(lst)):
            lst_.append([])
            for j in range(len(dict_)):
                lst_[i].append(0)

        for i,word in enumerate(list(dict_.keys())):
            for k1,v1 in dict_[word].items():
                lst_[k1][i] = v1
        self.features_list = dict_.keys() # сразу сделать лист
        
        #посмотреть Counter Collections defaultdict setdefault
        
        return [[tf*idf for tf,idf in zip(tf_row,self.idf_transform(lst_))] for tf_row in self.tf_transform(lst_)]
    
    def get_feature_names(self) -> list:
        '''
        Getting list of words
        '''
        return list(self.features_list)

In [88]:
corpus = [
 'Crock Pot Pasta Never boil pasta again',
 'Pasta Pomodoro Fresh ingredients Parmesan to taste'
]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(tfidf_matrix)

['crock', 'pot', 'pasta', 'never', 'boil', 'again', 'pomodoro', 'fresh', 'ingredients', 'parmesan', 'to', 'taste']
[[0.0416174424324285, 0.0416174424324285, 0.14, 0.0416174424324285, 0.0416174424324285, 0.0416174424324285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.07, 0.0, 0.0, 0.0, 0.0416174424324285, 0.0416174424324285, 0.0416174424324285, 0.0416174424324285, 0.0416174424324285, 0.0416174424324285]]
